In [1]:
import win32crypt

promptStruct = (0x1, 0, "hello")

eData = b'\x01\x00\x00\x00\xd0\x8c\x9d\xdf\x01\x15\xd1\x11\x8cz\x00\xc0O\xc2\x97\
\xeb\x01\x00\x00\x00\x01\x90\x01\xe7\xb1\x06+G\xb5\xe9\xba`l|\xe1}\x13\x00\x00\x00\
\x18\x00\x00\x00d\x00e\x00s\x00c\x00r\x00i\x00p\x00t\x00i\x00o\x00n\x00\x00\x00\x10f\
\x00\x00\x00\x01\x00\x00\x00\x00\x00\x07\xceeu\nZ\xe8\xc3e\x1eW\x10\x7f\xa14\x8c\xcc\
\x87G\x05UM\xbb=\xcal\xb7\xb2\x17\t\x14S\x00\x00\x00\x00\x0e\x80\x00\x00\x00\x02\x00\
\x00 \x00\x00\x00\xdav\xd2\xa4\xea\xae\xb6\xa6\xd9\xaf\x91Rbi\xf7q\x91\xb3\xa3\x9d\xeeS\
\x99!\xac8\xbeoHw\xfa_\x10\x00\x00\x008)7I\xfb\x96\xbb\xf6\x94\xdcJQ\x90\xaf\x15\x9b@\
\x00\x00\x00\xdc\xebbD\x80\x0b\x95\xc3\x13\xd1\xa8\x8d\x0fL~\x17\xf9\xbf\xc5dd\x94@g\
\x83>\x82\x13`\xc6i\xac\xb7\xcf\xfc\xc0\x19=)\xab\xf6P\xaf\x83\x184#g\xe8\x8d\x93\x89\
\xd1B\xc6\x02\x9b\x9f.@b\xfc\xd7\x8d'
#CryptProtectData 시 암호화된 데이터

dData = win32crypt.CryptUnprotectData(eData, None, None, promptStruct, 0)
print(f"{dData}")

error: (-2146893813, 'CryptUnprotectData', '키를 지정된 상태에서 사용하기에 부적합합니다.')

In [17]:
import os
import sqlite3
import win32crypt
import json
import base64
from Cryptodome.Cipher import AES
import shutil

def get_master_key():
    with open(os.environ['USERPROFILE'] + os.sep + r'AppData\Local\Google\Chrome\User Data\Local State', "r",encoding='utf-8') as f:
        local_state = f.read()
        local_state = json.loads(local_state) #joson load
    master_key = base64.b64decode(local_state["os_crypt"]["encrypted_key"])
    master_key = master_key[5:]  # removing DPAPI
    master_key = win32crypt.CryptUnprotectData(master_key, None, None, None, 0)[1]

    return master_key

def decrypt_payload(cipher, payload):
    return cipher.decrypt(payload)


def generate_chipher(aes_key, iv):
    return AES.new(aes_key, AES.MODE_GCM, iv)

def decrypt_password(buff, master_key):
    try:
        iv = buff[3:15]
        payload = buff[15:-16]
        cipher = generate_chipher(master_key, iv)
        decrypt_pass = decrypt_payload(cipher, payload)
        decrypt_pass = decrypt_pass.decode('utf-8')  # remove suffix bytes.
        return decrypt_pass
    except Exception as e:
        print("Probably saved password from Chrome version older than v80\n")
        print(str(e))
        return "Chrome < 80"


master_key = get_master_key()
print('[+] Master Key :',master_key)
login_db = os.environ['USERPROFILE'] + os.sep + r'AppData\Local\Google\Chrome\User Data\default\Login Data For Account'
shutil.copy2(login_db, "Loginvault.db") #making a temp copy since Login Data DB is locked while Chrome is running

#connect to sqlite database
conn = sqlite3.connect("Loginvault.db")
cursor = conn.cursor()


try:
    cursor.execute("SELECT origin_url, username_value, password_value FROM Logins")
    for index, login in enumerate(cursor.fetchall()):
        url = login[0]
        username = login[1]
        ciphertext = login[2]
        decrypted_password = decrypt_password(ciphertext, master_key)
        if len(url) > 0:
            print("URL: " + url + "\nUser Name: " + username + "\nPassword: " + decrypted_password + "\n" + "*" * 50 + "\n")
except Exception as e:
    print(e)
    pass

cursor.close()
conn.close()

try:
    os.remove("Loginvault.db")

except Exception as e:
    print(e)
    pass

[+] Master Key : b'\xc7z\xb2\x1b\xe3\xa7d\xb7H\x04H\xbc4T\xa0p\x10\x98\x97\x9bP\xe7\x80^\x1d\xb3\x94\x99\xa1\xe9\xb1\x9d'
